#**Part I. Creating a head**

In [ ]:
!git clone https://github.com/AmarSaini/Epoching_StyleGan2_Setup.git

In [2]:
import os

os.chdir('/content/Epoching_StyleGan2_Setup')

In [ ]:
!git clone https://github.com/cleardusk/3DDFA.git

In [ ]:
!pip install requests
!pip install Pillow
!pip install dlib
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15

In [6]:
import sys
sys.path.append('stylegan2/')

from stylegan2 import pretrained_networks
from stylegan2 import dnnlib
from stylegan2.dnnlib import tflib

from pathlib import Path
from PIL import Image
import pickle
import numpy as np

import ipywidgets as widgets

model_path = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
results_size = 1024

In [7]:
# Code to load the StyleGAN2 Model
def load_model():
    _G, _D, Gs = pretrained_networks.load_networks(model_path)
    
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    
    return Gs, noise_vars, Gs_kwargs

# Generate images given a random seed (Integer)
def generate_image_random(rand_seed):
    rnd = np.random.RandomState(rand_seed)
    z = rnd.randn(1, *Gs.input_shape[1:])
    tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars})
    images = Gs.run(z, None, **Gs_kwargs)
    return images, z

# Generate images given a latent code ( vector of size [1, 512] )
def generate_image_from_z(z):
    images = Gs.run(z, None, **Gs_kwargs)
    return images

In [ ]:
Gs, noise_vars, Gs_kwargs = load_model()

In [9]:
os.mkdir('images/')

In [ ]:
# Creating random head

random_num = np.random.randint(0, 4294967295)
image, latent_code1 = generate_image_random(random_num)
image = Image.fromarray(image[0]).resize((results_size, results_size))
image.save('/content/Epoching_StyleGan2_Setup/images/image.jpg')
image

#**Part II. Changing facial attributes**

In [11]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'restart': True, 'status': 'ok'}

In [1]:
import sys
import os

os.chdir('/content/Epoching_StyleGan2_Setup')

sys.path.append('stylegan2/')

from stylegan2 import pretrained_networks
from stylegan2 import dnnlib
from stylegan2.dnnlib import tflib

from pathlib import Path
from PIL import Image
import pickle
import numpy as np

import ipywidgets as widgets

model_path = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
results_size = 1024

In [ ]:
!python -W ignore stylegan2/dataset_tool.py create_from_images datasets_stylegan2/custom_imgs images/

In [ ]:
!python -W ignore stylegan2/epoching_custom_run_projector.py project-real-images --network=$model_path \
  --dataset=custom_imgs --data-dir=datasets_stylegan2 --num-images=1 --num-snapshots 100

In [4]:
def load_model():
    _G, _D, Gs = pretrained_networks.load_networks(model_path)
    
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    
    return Gs, noise_vars, Gs_kwargs


def get_final_latents():
    all_results = list(Path('results/').iterdir())
    print(all_results)
    all_results.sort()
    
    last_result = all_results[-1]

    latent_files = [x for x in last_result.iterdir() if 'final_latent_code' in x.name]
    latent_files.sort()
    
    all_final_latents = []
    
    for file in latent_files:
        with open(file, mode='rb') as latent_pickle:
            all_final_latents.append(pickle.load(latent_pickle))
    
    return all_final_latents


def get_control_latent_vectors(path):
    files = [x for x in Path(path).iterdir() if str(x).endswith('.npy')]
    latent_vectors = {f.name[:-4]:np.load(f) for f in files}
    return latent_vectors


def generate_image_from_projected_latents(latent_vector):
    images = Gs.components.synthesis.run(latent_vector, **Gs_kwargs)
    return images

In [ ]:
Gs, noise_vars, Gs_kwargs = load_model()

In [ ]:
latent_codes = get_final_latents()
# len(latent_codes), latent_codes[0].shape

In [ ]:
latent_controls = get_control_latent_vectors('stylegan2directions/')
del latent_controls['roll']
del latent_controls['yaw']
del latent_controls['pitch']

latent_controls.keys()

In [11]:
def apply_latent_controls(self):
    
    image_outputs = controller.children[0]
    feature_sliders = controller.children[1]
    
    slider_hboxes = feature_sliders.children[:-2]
    latent_movements = [(x.children[1].value, x.children[0].value) for x in slider_hboxes]

    modified_latent_code = np.array(latent_code_to_use)
    for feature, amount_to_move in latent_movements:
        modified_latent_code += latent_controls[feature]*amount_to_move

    images = generate_image_from_projected_latents(modified_latent_code)
    latent_img = Image.fromarray(images[0]).resize((1024, 1024))
    
    latent_img_output = image_outputs.children[1]
    latent_img.save('/content/Epoching_StyleGan2_Setup/3DDFA/samples/image.jpg')

    with latent_img_output:
        latent_img_output.clear_output()
        display(latent_img)
    

def reset_latent_controls(self):
    
    image_outputs = controller.children[0]
    feature_sliders = controller.children[1]
    
    slider_hboxes = feature_sliders.children[:-2]
    for x in slider_hboxes:
        x.children[0].value = 0
        
    latent_img_output = image_outputs.children[1]
    with latent_img_output:
        latent_img_output.clear_output()
        display(image_to_use)

def create_interactive_latent_controller():
    orig_img_output = widgets.Output()

    with orig_img_output:
        orig_img_output.clear_output()
        display(image_to_use)

    latent_img_output = widgets.Output()

    with latent_img_output:
        latent_img_output.clear_output()
        display(image_to_use)

    image_outputs = widgets.VBox([orig_img_output, latent_img_output])

    generate_button = widgets.Button(description='Generate', layout=widgets.Layout(width='50%', height='3%'))
    generate_button.on_click(apply_latent_controls)

    reset_button = widgets.Button(description='Reset Latent Controls', layout=widgets.Layout(width='50%', height='3%'))
    reset_button.on_click(reset_latent_controls)

    feature_sliders = []
    for feature in latent_controls:
        label = widgets.Label(feature)
        slider = widgets.FloatSlider(min=-25, max=25)
        feature_sliders.append(widgets.HBox([slider, label]))
    feature_sliders.append(generate_button)
    feature_sliders.append(reset_button)
    # feature_sliders.append(save_button)

    feature_sliders = widgets.VBox(feature_sliders)

    return widgets.HBox([image_outputs, feature_sliders])

In [ ]:
image = Image.open('/content/Epoching_StyleGan2_Setup/images/image.jpg')

latent_code_to_use = latent_codes[0]
image_to_use = image

controller = create_interactive_latent_controller()
controller

#**Creating 3D model and dataset with different poses**

In [15]:
os.chdir('/content/Epoching_StyleGan2_Setup/3DDFA/utils/cython')

In [ ]:
!python3 setup.py build_ext -i

##Download model from [here](https://drive.google.com/file/d/1kxgOZSds1HuUIlvo5sRH3PJv377qZAkE/view) and put it into '/content/Epoching_StyleGan2_Setup/3DDFA/models'


In [17]:
os.chdir('/content/Epoching_StyleGan2_Setup/3DDFA')

In [ ]:
!python main.py -f samples/image.jpg

In [ ]:
!pip install trimesh

In [21]:
import trimesh
# import numpy as np

In [23]:
face = trimesh.load('/content/Epoching_StyleGan2_Setup/3DDFA/samples/image_0.obj', force='mesh' )
face.invert()

In [ ]:
#   Rotate Z-coordinate (ROLL)
alpha = np.radians(-90)

rotation_matrix = np.array([[np.cos(alpha),  -np.sin(alpha),       0,        0],
                            [np.sin(alpha),   np.cos(alpha),       0,        0],
                            [            0,               0,       1,        0],
                            [            0,               0,       0,        1]])

face.apply_transform(rotation_matrix)
face.export('/content/result.obj')
face.show()

In [25]:
face = trimesh.load('/content/result.obj')
os.mkdir('/content/results/')

In [26]:
start_radian = -30
end_radian = 30
step = 10

def apply_roll_rotate(angle, face):
    alpha = np.radians(angle)
    rotation_matrix = np.array([[np.cos(alpha),  -np.sin(alpha),       0,        0],
                                [np.sin(alpha),   np.cos(alpha),       0,        0],
                                [            0,               0,       1,        0],
                                [            0,               0,       0,        1]])
    
    face.apply_transform(rotation_matrix)

def apply_yaw_rotate(angle, face):
    alpha = np.radians(angle)
    rotation_matrix = np.array([[ np.cos(alpha),               0,       np.sin(alpha),        0],
                                [             0,               1,                   0,        0],
                                [-np.sin(alpha),               0,       np.cos(alpha),        0],
                                [             0,               0,                   0,        1]])
    
    face.apply_transform(rotation_matrix)

def apply_pitch_rotate(angle, face):
    alpha = np.radians(angle)
    rotation_matrix = np.array([[1,                 0,                   0,        0],
                                [0,     np.cos(alpha),      -np.sin(alpha),        0],
                                [0,     np.sin(alpha),       np.cos(alpha),        0],
                                [0,                 0,                   0,        1]])
    
    face.apply_transform(rotation_matrix)

In [27]:
for angle_yaw in range(start_radian, end_radian, step):
    for angle_pitch in range(start_radian, end_radian, step):
        for angle_roll in range(start_radian, end_radian, step):
            face1 = face.copy()
            apply_yaw_rotate(angle_yaw, face1)
            apply_pitch_rotate(angle_pitch, face1)
            apply_roll_rotate(angle_roll, face1)
            face1.export(f'/content/results/{angle_yaw}_{angle_pitch}_{angle_roll}_rotate.obj')

In [ ]:
!zip -r /content/dataset.zip /content/results

In [ ]:
from google.colab import files
files.download("/content/dataset.zip")